# AN2DL - Second Challenge

## Initial Operations

### Import the libraries

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers

### Set seed for reproducibility

In [ ]:
seed = 69
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Load the dataset

In [ ]:
dataset = pd.read_csv('training.csv')
print(dataset.shape)
dataset.head()

(68528, 7)


,Sponginess,Wonder level,Crunchiness,Loudness on impact,Meme creativity,Soap slipperiness,Hype root
0,7.97698,4.33494,10.67282,1.76692,3.22440,51.681460,3.65434
1,8.07824,4.44616,10.56160,1.70716,3.32566,51.563598,3.47672
2,8.02844,4.22372,10.56160,1.64906,3.17460,50.863080,3.47672
3,8.02844,4.22372,10.56160,1.70716,3.17460,45.841581,3.47672
4,7.87572,4.44616,10.45038,1.70716,3.27586,47.126421,3.47672


## Model (Direct Forecast)

### Normalization

In [ ]:
X_train_raw = dataset.copy()

# Normalize both train and test samples
X_min = X_train_raw.min()
X_max = X_train_raw.max()
X_train_raw = (X_train_raw-X_min)/(X_max-X_min)

### Define sequences

In [ ]:
target_labels = dataset.columns
window = 1000
stride = 50
telescope = 864

In [ ]:
def build_sequences(df, target_labels=['pollution'], window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)

### Define the model

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 200

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    convlstm = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(input_layer)
    convlstm = tfkl.Conv1D(128, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.MaxPool1D()(convlstm)
    convlstm = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True))(convlstm)
    convlstm = tfkl.Conv1D(256, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.5)(convlstm)

    # In order to predict the next values for more than one channel,
    # we can use a Dense layer with a number given by telescope*num_channels,
    # followed by a Reshape layer to obtain a tensor of dimension 
    # [None, telescope, num_channels]
    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu')(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same')(output_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae'])

    # Return the model
    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 1000, 7)]         0         
                                                                 
 bidirectional (Bidirectiona  (None, 1000, 128)        36864     
 l)                                                              
                                                                 
 conv1d (Conv1D)             (None, 1000, 128)         49280     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 500, 128)         0         
 )                                                               
                                                                 
 bidirectional_1 (Bidirectio  (None, 500, 256)         263168    
 nal)                                                            
                                                             

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

Epoch 1/200
19/19 [==============================] - 26s 625ms/step - loss: 0.1740 - mae: 0.3402 - val_loss: 0.1132 - val_mae: 0.2654 - lr: 0.0010
Epoch 2/200
19/19 [==============================] - 9s 501ms/step - loss: 0.1113 - mae: 0.2611 - val_loss: 0.0934 - val_mae: 0.2406 - lr: 0.0010
Epoch 3/200
19/19 [==============================] - 9s 498ms/step - loss: 0.0911 - mae: 0.2355 - val_loss: 0.0758 - val_mae: 0.2138 - lr: 0.0010
Epoch 4/200
19/19 [==============================] - 9s 501ms/step - loss: 0.0734 - mae: 0.2107 - val_loss: 0.0623 - val_mae: 0.1944 - lr: 0.0010
Epoch 5/200
19/19 [==============================] - 9s 500ms/step - loss: 0.0581 - mae: 0.1878 - val_loss: 0.0493 - val_mae: 0.1679 - lr: 0.0010
Epoch 6/200
19/19 [==============================] - 9s 500ms/step - loss: 0.0460 - mae: 0.1676 - val_loss: 0.0402 - val_mae: 0.1513 - lr: 0.0010
Epoch 7/200
19/19 [==============================] - 9s 500ms/step - loss: 0.0367 - mae: 0.1498 - val_loss: 0.0338 - val_ma

### Save the model

In [ ]:
model.save("Direct Forecast")

In [ ]:
X = tf.constant(dataset) # This is NOT for submission
X = X.numpy()
future = X[-window:]
X_min = X.min(axis=0)
X_max = X.max(axis=0)
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future,0)
out = model.predict(future)
out = np.squeeze(out,0)
out = out * (X_max - X_min) + X_min
out = tf.convert_to_tensor(out)

(864, 7)
